In [1]:
# fetching 2 files from bbsn00

In [2]:
import os
import json
import numpy as np
import pandas as pd

In [3]:
os.makedirs('data', exist_ok=True)

In [4]:
!scp tuplex@bbsn00:/disk/download/data/2011-02-12-10.json.gz data/

2011-02-12-10.json.gz                         100%  181KB 229.0KB/s   00:00    


In [5]:
!scp tuplex@bbsn00:/disk/download/data/2021-02-12-10.json.gz data/

2021-02-12-10.json.gz                         100%   67MB   2.9MB/s   00:23    


In [64]:
!scp tuplex@bbsn00:/disk/download/data/2015-06-05-10.json.gz data/

2015-06-05-10.json.gz                         100% 9137KB   2.2MB/s   00:04    


In [65]:
!ls data

2011-02-12-10.json    2015-06-05-10.json.gz 2021-02-12-10.json.gz
2011-02-12-10.json.gz 2021-02-12-10.json


In [8]:
!wc -l data/2021-02-12-10.json

  120737 data/2021-02-12-10.json


In [9]:
!wc -l data/2011-02-12-10.json

    1057 data/2011-02-12-10.json


In [66]:
!wc -l data/2015-06-05-10.json.gz

   34099 data/2015-06-05-10.json.gz


Observation 1:
Amount of data changed a lot. Therefore good idea to process differently? I.e., did average #keys/#length change?

In [4]:
with open('data/2021-02-12-10.json') as fp:
    lines = fp.readlines()

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [5]:
def get_global_stats(path):
    with open(path) as fp:
        lines = fp.readlines()
    
    n_lines = len(lines)
    n_length = sum(map(lambda x: len(x.strip()), lines))
    n_key_lengths = np.array([len(json.loads(line).keys()) for line in lines])
    avg_nkeys = n_key_lengths.mean()
    min_nkeys = n_key_lengths.min()
    max_nkeys = n_key_lengths.max()
    uniq_nkeys, key_counts = np.unique(n_key_lengths, return_counts=True)
    key_counts = key_counts.astype(np.float64) / n_lines
    uniq_nkeys = dict(zip(list(uniq_nkeys), list(key_counts)))
    
    # count 2nd level keys!
    def get_2nd_level_keys(line):
        d = json.loads(line)
        n = 0
        for v in d.values():
            if isinstance(v, dict):
                n += len(v.keys())
            else:
                n += 1
        return n
    
    n_2key_lengths = np.array([get_2nd_level_keys(line) for line in lines])
    u, c = np.unique(n_2key_lengths, return_counts=True)
    c = key_counts.astype(np.float64) / n_lines
    uniq_2_keys = dict(zip(list(u), list(c)))
    
    return {'nlines' : n_lines,
            'avg_line_length' : n_length / n_lines,
            'path' : path,
           'avg_nkeys' : avg_nkeys,
           'min_nkeys' : min_nkeys,
           'max_nkeys' : max_nkeys,
           'nunique_1st_level' : uniq_nkeys,
           'nunique_2nd_level': uniq_2_keys}

get_global_stats('data/2011-02-12-10.json')

{'nlines': 1057,
 'avg_line_length': 881.2147587511826,
 'path': 'data/2011-02-12-10.json',
 'avg_nkeys': 7.114474929044466,
 'min_nkeys': 7,
 'max_nkeys': 8,
 'nunique_1st_level': {7: 0.8855250709555346, 8: 0.11447492904446546},
 'nunique_2nd_level': {11: 0.0008377720633448766, 13: 0.00010830173041103639}}

In [6]:
with open('data/2021-02-12-10.json') as fp:
    lines = fp.readlines()

In [10]:
event_type = np.array([json.loads(line)['type'] for line in lines])
u, c = np.unique(event_type, return_counts=True)
c = c.astype(np.float64) / len(event_type)
dict(zip(u, c))

{'CommitCommentEvent': 0.005321802760685182,
 'CreateEvent': 0.12032263429236655,
 'DeleteEvent': 0.02141194079494429,
 'ForkEvent': 0.033552303342757356,
 'GollumEvent': 0.011807749875270248,
 'IssueCommentEvent': 0.09504407117911193,
 'IssuesEvent': 0.04652419757192749,
 'MemberEvent': 0.004032928654581739,
 'PublicEvent': 0.0010394146016963247,
 'PullRequestEvent': 0.055380009978380176,
 'PullRequestReviewCommentEvent': 0.017919507733244636,
 'PushEvent': 0.4936387826376185,
 'ReleaseEvent': 0.0035340096457675036,
 'WatchEvent': 0.0904706469316481}

In [11]:
with open('data/2011-02-12-10.json') as fp:
    lines = fp.readlines()
    
    
event_type = np.array([json.loads(line)['type'] for line in lines])
u, c = np.unique(event_type, return_counts=True)
c = c.astype(np.float64) / len(event_type)
dict(zip(u, c))

{'CommitCommentEvent': 0.003784295175023652,
 'CreateEvent': 0.1901608325449385,
 'DeleteEvent': 0.010406811731315043,
 'DownloadEvent': 0.000946073793755913,
 'FollowEvent': 0.039735099337748346,
 'ForkEvent': 0.039735099337748346,
 'GistEvent': 0.03595080416272469,
 'GollumEvent': 0.01229895931882687,
 'IssuesEvent': 0.03595080416272469,
 'MemberEvent': 0.006622516556291391,
 'PullRequestEvent': 0.019867549668874173,
 'PushEvent': 0.4966887417218543,
 'WatchEvent': 0.10785241248817408}

In [12]:
with open('data/2015-06-05-10.json') as fp:
    lines = fp.readlines()
    
    
event_type = np.array([json.loads(line)['type'] for line in lines])
u, c = np.unique(event_type, return_counts=True)
c = c.astype(np.float64) / len(event_type)
dict(zip(u, c))

{'CommitCommentEvent': 0.005321802760685182,
 'CreateEvent': 0.12032263429236655,
 'DeleteEvent': 0.02141194079494429,
 'ForkEvent': 0.033552303342757356,
 'GollumEvent': 0.011807749875270248,
 'IssueCommentEvent': 0.09504407117911193,
 'IssuesEvent': 0.04652419757192749,
 'MemberEvent': 0.004032928654581739,
 'PublicEvent': 0.0010394146016963247,
 'PullRequestEvent': 0.055380009978380176,
 'PullRequestReviewCommentEvent': 0.017919507733244636,
 'PushEvent': 0.4936387826376185,
 'ReleaseEvent': 0.0035340096457675036,
 'WatchEvent': 0.0904706469316481}

Let's investigate PushEvents (since they're the majority!)

In [20]:
path = 'data/2015-06-05-10.json'
path = 'data/2021-02-12-10.json'
with open(path) as fp:
    lines = fp.readlines()
    
    rows = [json.loads(line) for line in lines]

In [21]:
rows_2 = list(filter(lambda x: x['type'] == 'PushEvent', rows))

In [22]:
rows_2[10]

{'id': '15162035923',
 'type': 'PushEvent',
 'actor': {'id': 20182680,
  'login': 'LinuxServer-CI',
  'display_login': 'LinuxServer-CI',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/LinuxServer-CI',
  'avatar_url': 'https://avatars.githubusercontent.com/u/20182680?'},
 'repo': {'id': 87807234,
  'name': 'linuxserver/docker-duplicati',
  'url': 'https://api.github.com/repos/linuxserver/docker-duplicati'},
 'payload': {'push_id': 6521264030,
  'size': 1,
  'distinct_size': 1,
  'ref': 'refs/heads/development',
  'head': '9828bf14dc79c44ba1754e8d3aa7a294df0c48d9',
  'before': '58c191a3b0bf5aa8b96b5f2b17c232f81b784b9b',
  'commits': [{'sha': '9828bf14dc79c44ba1754e8d3aa7a294df0c48d9',
    'author': {'name': 'LinuxServer-CI',
     'email': '5a4fe08359c7f97380e408c717ef42c86939cd86@linuxserver.io'},
    'message': 'Bot Updating Templated Files',
    'distinct': True,
    'url': 'https://api.github.com/repos/linuxserver/docker-duplicati/commits/9828bf14dc79c44ba1754e8d3aa7a294d

In [28]:
rows_2[10]['org']

{'id': 12324908,
 'login': 'linuxserver',
 'gravatar_id': '',
 'url': 'https://api.github.com/orgs/linuxserver',
 'avatar_url': 'https://avatars.githubusercontent.com/u/12324908?'}

In [30]:
n_org_pushes = len(list(filter(lambda x: x['type'] == 'PushEvent' and 'org' in x.keys(), rows)))
n_org_pushes, len(rows_2)

(13383, 56592)

Maybe different org distribution? Could this lead to a difference?

In [17]:
rows_2[10]

{'id': '2866927111',
 'type': 'PushEvent',
 'actor': {'id': 9982176,
  'login': 'roxyavgfli',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/roxyavgfli',
  'avatar_url': 'https://avatars.githubusercontent.com/u/9982176?'},
 'repo': {'id': 36805194,
  'name': 'roxyavgfli/Light_Bot_Project_Grp6',
  'url': 'https://api.github.com/repos/roxyavgfli/Light_Bot_Project_Grp6'},
 'payload': {'push_id': 686101578,
  'size': 3,
  'distinct_size': 3,
  'ref': 'refs/heads/master',
  'head': '2bd8b6c87a7087f5e9e870edeb8c73fb678d431b',
  'before': '3e14147cfab02a608172b01e19e7cbcc55015e26',
  'commits': [{'sha': 'a8011d422740912bd635e8c8b42c757f6de4dc85',
    'author': {'email': '093c1c14ef90f2c50d86cf4b78c754e6ba763b38@gmail.com',
     'name': 'Vial-Grelier Aymeric'},
    'message': "Merge branch 'master' into aym-dev",
    'distinct': True,
    'url': 'https://api.github.com/repos/roxyavgfli/Light_Bot_Project_Grp6/commits/a8011d422740912bd635e8c8b42c757f6de4dc85'},
   {'sha': 'b09de66e0

Query:

When users from which country usually fork your work? Timewise?
    
Query: 

Create fork/star network out of users - per month to show change? (requires groupby)

Query:

What are the most popular English swearwords on Github? Did swearword use decrease over the years?

Query:

How long do PRs usually take to get merged? Is there a correlation between larger repos/bigger project and speed?

## Check how the event types differ in frequency for the different github files. I.e., is there a pattern?

In [48]:
line = lines[0]

json.loads(line)['payload']

{'forkee': {'id': 338280635,
  'node_id': 'MDEwOlJlcG9zaXRvcnkzMzgyODA2MzU=',
  'name': 'OnlineClassesBot',
  'full_name': 'PROHackerr/OnlineClassesBot',
  'private': False,
  'owner': {'login': 'PROHackerr',
   'id': 75305464,
   'node_id': 'MDQ6VXNlcjc1MzA1NDY0',
   'avatar_url': 'https://avatars.githubusercontent.com/u/75305464?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/PROHackerr',
   'html_url': 'https://github.com/PROHackerr',
   'followers_url': 'https://api.github.com/users/PROHackerr/followers',
   'following_url': 'https://api.github.com/users/PROHackerr/following{/other_user}',
   'gists_url': 'https://api.github.com/users/PROHackerr/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/PROHackerr/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/PROHackerr/subscriptions',
   'organizations_url': 'https://api.github.com/users/PROHackerr/orgs',
   'repos_url': 'https://api.github.com/users/PROHackerr/repos',
  

In [43]:
 def get_2nd_level_keys(line):
        d = json.loads(line)
        n = 0
        for v in d.values():
            if isinstance(v, dict):
                n += len(v.keys())
            else:
                n += 1
        return n
    
get_2nd_level_keys(line)

14

In [36]:
rows = []
rows.append(get_global_stats('data/2021-02-12-10.json'))
rows.append(get_global_stats('data/2011-02-12-10.json'))
rows

[{'nlines': 120737,
  'avg_line_length': 4567.872781334637,
  'path': 'data/2021-02-12-10.json',
  'avg_nkeys': 7.352162137538617,
  'min_nkeys': 7,
  'max_nkeys': 8,
  'nuniq_1st_level': {7: 0.647837862461383, 8: 0.35216213753861697}},
 {'nlines': 1057,
  'avg_line_length': 881.2147587511826,
  'path': 'data/2011-02-12-10.json',
  'avg_nkeys': 7.114474929044466,
  'min_nkeys': 7,
  'max_nkeys': 8,
  'nuniq_1st_level': {7: 0.8855250709555346, 8: 0.11447492904446546}}]